In [1]:
%load_ext autoreload
%autoreload 2

# Simple RAG (Retrieval-Augmented Generation) System
## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying PDF documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

## Key Components

1. PDF processing and text extraction
2. Text chunking for manageable processing
3. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
4. Retriever setup for querying the processed documents
5. Evaluation of the RAG system

### Import libraries and environment variables

In [2]:
import os
import sys

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [4]:
from helpers.helper_functions import *
from helpers.evalute_rag import *

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

/tmp/ipykernel_9170/1451311806.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helpers.helper_functions import *
/home/stackops/.local/lib/python3.10/site-packages/deepeval/__init__.py:53: UserWarning: You are using deepeval version 2.3.7, however version 2.4.1 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### Read Docs

In [5]:
paths = ("/home/stackops/langchain-labs/data/vks/pdf/vi/01-vks-la-gi.pdf", 
         "/home/stackops/langchain-labs/data/vks/pdf/vi/02-nlb.pdf",
         "/home/stackops/langchain-labs/data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf",
         "/home/stackops/langchain-labs/data/vks/pdf/vi/04-thong-bao-cap-nhat-release-note.pdf")

### Encode document

In [6]:
def encode_pdf(paths: List[str], chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    cleaned_texts = []
    for path in paths:
        loader = PyPDFLoader(path)
        documents = loader.load()

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
        )
        texts = text_splitter.split_documents(documents)
        cleaned_texts = cleaned_texts + replace_t_with_space(texts)

    # Create embeddings (Tested with OpenAI and Amazon Bedrock)
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.HUGGINGFACE)
    #embeddings = get_langchain_embedding_provider(EmbeddingProvider.AMAZON_BEDROCK)

    # Create vector store
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [7]:
chunks_vector_store = encode_pdf(paths, chunk_size=8000, chunk_overlap=7800)

/home/stackops/.local/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/home/stackops/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create retriever

In [8]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 3})

### Test retriever

In [9]:
test_query = "NLB là gì?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
2. Tích hợp Network Load Balancer (NLB) trên VKS
2.1. Network Load Balancer (NLB) là gì?
Network Load Balancer (tên viết tắt là NLB) là một bộ cân bằng tải được cung cấp bởi VNGCloud giúp phân phối lưu lượng truy cập
mạng đến nhiều máy chủ back-end (backend servers) trong một nhóm máy tính (instance group). NLB hoạt động ở layer 4 của mô hình
OSI, giúp cân bằng tải dựa trên địa chỉ IP và cổng TCP/UDP. Để biết thêm thông tin chi tiết về NLB, vui lòng tham khảo tại
https://docs.vngcloud.vn/vng-cloud-document/vn/vks/network/lam-viec-voi-network-load-balancing-nlb
2.2. Mô hình triển khai
(Đường dẫn đến hình ảnh: https://docs.vngcloud.vn/~gitbook/image?url=https%3A%2F%2F3672463924-
ﬁles.gitbook.io%2F%7E%2Fﬁles%2Fv0%2Fb%2Fgitbook-x-
prod.appspot.com%2Fo%2Fspaces%252FB0NrrrdJdpYOYzRkbWp5%252Fuploads%252FVYBtJjEoUNgDi1f5J9vL%252Fimage.png%3Falt%3
Dmedia%26token%3D554a2d62-320e-48d1-a884-3c7cce589071&width=768&dpr=2&quality=100&sign=4336a109&sv=2)
VNGCloud LoadBalancer Controller: là

In [10]:
test_query = "Public cluster là gì, hãy so sánh nó với private cluster"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Public cluster: Khó kiểm soát hơn, có thể quản lý truy cập thông qua tính năng
Whitelist.
Private cluster: Kiểm soát truy cập chặt chẽ, mọi kết nối đều nằm trong mạng private
của VNG Cloud, từ đó giảm thiểu nguy cơ từ các cuộc tấn công mạng từ bên ngoài.
Khả năng mở rộng (AutoScaling):
Public cluster: Dễ dàng mở rộng thông qua tính năng Auto Scaling.
Private cluster: Dễ dàng mở rộng thông qua tính năng Auto Scaling.
Khả năng tự hồi phục (AutoHealing):
Public cluster: Tự động phát hiện lỗi và khởi động lại node (Auto Healing)
Private cluster: Tự động phát hiện lỗi và khởi động lại node (Auto Healing)
Khả năng truy cập từ bên ngoài:
Public cluster: Dễ dàng truy cập từ bất kỳ đâu với
Private cluster: Truy cập từ bên ngoài phải qua các giải pháp bảo mật khác.
Cấu hình và triển khai:
Public cluster: Đơn giản hơn do không yêu cầu thiết lập mạng nội bộ.
Private cluster: Phức tạp hơn, yêu cầu cấu hình mạng private và bảo mật.
Chi phí:
Public cluster: Thường thấp hơn do không cần thi

### Evaluate results

In [11]:
#Note - this currently works with OPENAI only
evaluate_rag(chunks_query_retriever, 1)

/home/stackops/langchain-labs/rag-techniques/helpers/evalute_rag.py:120: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(question)


{'questions': ['##  How might the changing climate impact the cultural practices and traditions of indigenous communities in the Arctic, and what steps can be taken to mitigate these impacts? ',
  ''],
 'results': ['```json\n{\n  "Relevance": 1,\n  "Completeness": 1,\n  "Conciseness": 1\n}\n```\n\nThe retrieved information is completely irrelevant to the question. It focuses on configuring Network Load Balancers (NLB) on VNGCloud, a cloud platform, which has no relation to the impact of climate change on indigenous communities in the Arctic. \n\nThe context is not complete as it does not provide any information related to the question. It is also not concise as it provides a detailed explanation of NLB configuration on VNGCloud, which is not relevant to the user\'s query. \n',
  '```json\n{\n  "Relevance": 3,\n  "Completeness": 2,\n  "Conciseness": 2\n}\n```\n\n**Explanation:**\n\n* **Relevance:** The retrieved context is somewhat relevant as it discusses Network Load Balancer (NLB) on